In [1]:
from pathlib import Path


PROJECT_ROOT_DIR = str(Path.cwd().parent)
PROJECT_ROOT_DIR

'/root/llm_graph_embedder'

# Exploring Gemma 2B-it model

In [2]:
device = "cuda"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%time

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", device_map="cuda")
# model = AutoModel.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", device_map="cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 34.2 s, sys: 15.1 s, total: 49.3 s
Wall time: 12 s


In [5]:
model.to(device)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=

# Loading data

In [6]:
import pandas as pd


train_data = pd.read_parquet(f"{PROJECT_ROOT_DIR}/dataset/train.parquet").sample(n=3)
eval_data = pd.read_parquet(f"{PROJECT_ROOT_DIR}/dataset/eval.parquet").sample(n=1)
test_data = pd.read_parquet(f"{PROJECT_ROOT_DIR}/dataset/test.parquet").sample(n=1)

In [7]:
test_data["formatted_conversations"].tolist()[0]

array([{'from': 'human', 'value': 'Given a citation graph: \n{\'edge_index\': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 2, 36, 8, 37, 0, 38, 39, 40, 41, 42, 43, 44, 45, 46, 0, 1, 2, 3, 8, 9, 0, 47, 48, 0, 49, 50, 51, 52, 14, 53, 33, 2, 1, 54, 55, 36, 56, 57, 54, 58, 59, 60, 61, 48, 62, 63, 30, 64], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]], \'node_idx\': 705, \'node_list\': [705, 2954, 13977, 40224, 86867, 109707, 121813, 122636, 148744, 150520, 83482, 36473, 147321, 149020, 11068, 95784, 85893, 110889, 134484, 87622, 99761, 53659, 106049, 139363, 132257, 40415, 53550, 120884, 20953, 43332, 6786, 28135, 92761, 80150, 131487, 97945, 156929, 14775, 59798, 4695, 67762, 104576, 105500, 67312, 13

In [8]:
def format_training_conversations(sample):
    prompt_template = """<human>: {human_turn}\n\n<gpt>: {gpt_turn}"""

    return prompt_template.format(
        human_turn=sample["formatted_conversations"][0]["value"],
        gpt_turn=sample["formatted_conversations"][1]["value"],
    )

def format_test_conversations(sample):
    prompt_template = """<human>: {human_turn}\n\n<gpt>: """

    return prompt_template.format(
        human_turn=sample["formatted_conversations"][0]["value"],
    )

In [9]:
train_data["full_conversation"] = train_data.apply(format_training_conversations, axis=1)
eval_data["full_conversation"] = eval_data.apply(format_training_conversations, axis=1)

test_data["full_conversation"] = test_data.apply(format_test_conversations, axis=1)

In [20]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train_data[['full_conversation']].reset_index(drop=True)),
        "eval": Dataset.from_pandas(eval_data[['full_conversation']].reset_index(drop=True)),
        "test": Dataset.from_pandas(test_data[['full_conversation']].reset_index(drop=True)),
    }
)

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['full_conversation'],
        num_rows: 3
    })
    eval: Dataset({
        features: ['full_conversation'],
        num_rows: 1
    })
    test: Dataset({
        features: ['full_conversation'],
        num_rows: 1
    })
})

# Fine tune Gemma in a couple of samples

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["full_conversation"], padding="max_length", truncation=True, max_length=1024
    )


tokenized_train_data = dataset["train"].map(tokenize_function, batched=True)
tokenized_eval_data = dataset["eval"].map(tokenize_function, batched=True)

tokenized_train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask"]
)
tokenized_eval_data.set_format(
    type="torch", columns=["input_ids", "attention_mask"]
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [26]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
)

In [ ]:
trainer.train()